In [275]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests

In [2]:
endpoint_url = "https://api.coingecko.com/api/v3/ping"

params = {
    "vs_currency": "usd",
    "order": "market_cap_desc",
    "per_page": 10,
    "page": 1
}

reponse = requests.get(endpoint_url)

if reponse.status_code == 200:
    print(reponse.json())
else:
    print("Error: ", reponse.status_code)

{'gecko_says': '(V3) To the Moon!'}


# Having a look at CoinGecko API

An API requires an endpoint direcoty and maybe parameters. Tha's not the case for the end point currencies. We just need the available currencies

#### Coins


Gecko offers literally tons of different coins (10k+)... which is kinda crazy, so let's only focus on the treding to only pick 10 of them.. and let's search what the end point `search/trending` has for us

In [285]:
coingecko = "https://api.coingecko.com/api/v3/"
trend_coins_ep = "search/trending"

trend_coins = requests.get(coingecko + trend_coins_ep).json()
trend_coins

{'coins': [{'item': {'id': 'sei-network',
    'coin_id': 28205,
    'name': 'Sei',
    'symbol': 'SEI',
    'market_cap_rank': 99,
    'thumb': 'https://assets.coingecko.com/coins/images/28205/thumb/TKmaTqMv_400x400.jpeg?1668395911',
    'small': 'https://assets.coingecko.com/coins/images/28205/small/TKmaTqMv_400x400.jpeg?1668395911',
    'large': 'https://assets.coingecko.com/coins/images/28205/large/TKmaTqMv_400x400.jpeg?1668395911',
    'slug': 'sei',
    'price_btc': 8.176064445323926e-06,
    'score': 0}},
  {'item': {'id': 'cyberconnect',
    'coin_id': 31274,
    'name': 'Cyberconnect',
    'symbol': 'CYBER',
    'market_cap_rank': 378,
    'thumb': 'https://assets.coingecko.com/coins/images/31274/thumb/cyberconnect.png?1692101771',
    'small': 'https://assets.coingecko.com/coins/images/31274/small/cyberconnect.png?1692101771',
    'large': 'https://assets.coingecko.com/coins/images/31274/large/cyberconnect.png?1692101771',
    'slug': 'cyberconnect',
    'price_btc': 0.0001761

ok.. when I run into this messy view structures.. I just zoom into each level that compose the object. The throws a simple dictionary.. and let's just get the keys and see what we've got

In [286]:
trend_coins.keys()

dict_keys(['coins', 'nfts', 'exchanges'])

In [287]:
len(trend_coins["coins"]), len(trend_coins["nfts"]), len(trend_coins["exchanges"])

(7, 5, 0)

In [288]:
coin_list_df = pd.DataFrame(trend_coins["coins"]).rename(columns={"item": "coin_information"})
# Only for checkking the appropiate object --> dict, that's ok
type(coin_list_df["coin_information"][0]), coin_list_df["coin_information"][0]

(dict,
 {'id': 'sei-network',
  'coin_id': 28205,
  'name': 'Sei',
  'symbol': 'SEI',
  'market_cap_rank': 99,
  'thumb': 'https://assets.coingecko.com/coins/images/28205/thumb/TKmaTqMv_400x400.jpeg?1668395911',
  'small': 'https://assets.coingecko.com/coins/images/28205/small/TKmaTqMv_400x400.jpeg?1668395911',
  'large': 'https://assets.coingecko.com/coins/images/28205/large/TKmaTqMv_400x400.jpeg?1668395911',
  'slug': 'sei',
  'price_btc': 8.176064445323926e-06,
  'score': 0})

In [289]:
for key, value in coin_list_df["coin_information"][0].items():
    coin_list_df[key] = coin_list_df["coin_information"].apply(lambda coin: coin[key])


coin_list_df.drop(columns="coin_information", inplace=True)

In [290]:
coin_list_df["thumb"][0]

'https://assets.coingecko.com/coins/images/28205/thumb/TKmaTqMv_400x400.jpeg?1668395911'

In [291]:
coin_list_df.drop(columns=["thumb", "small", "large"], inplace=True)

In [292]:
coin_list_df

,id,coin_id,name,symbol,market_cap_rank,slug,price_btc,score
0,sei-network,28205,Sei,SEI,99,sei,0.000008,0
1,cyberconnect,31274,Cyberconnect,CYBER,378,cyberconnect,0.000176,1
2,thorchain,6595,THORChain,RUNE,84,thorchain,0.000055,2
3,apecoin,24383,ApeCoin,APE,62,apecoin,0.000063,3
4,lybra-finance,29958,Lybra Finance,LBR,537,lybra-finance,0.000077,4
5,harrypotterobamasonic10in,30323,HarryPotterObamaSonic10Inu (ETH),BITCOIN,183,harrypotterobamasonic10inu-eth,0.000006,5
6,worldcoin-wld,31069,Worldcoin,WLD,159,worldcoin,0.000054,6


In [293]:
",".join(coin_list_df["id"].tolist())

'sei-network,cyberconnect,thorchain,apecoin,lybra-finance,harrypotterobamasonic10in,worldcoin-wld'

In [294]:
search_coin_ep = "search"

search_coin_parameters = {
    "query": "coins"
}

requests.get(coingecko + search_coin_ep, params=search_coin_parameters).json()["coins"][0]

{'id': 'localcoinswap',
 'name': 'LocalCoinSwap',
 'api_symbol': 'localcoinswap',
 'symbol': 'LCS',
 'market_cap_rank': 1321,
 'thumb': 'https://assets.coingecko.com/coins/images/3998/thumb/LocalCoinSwap.png',
 'large': 'https://assets.coingecko.com/coins/images/3998/large/LocalCoinSwap.png'}

#### Getting currencies

`/simple/supported_vs_currencies` end point

In [295]:
supported_currencies_ep = "simple/supported_vs_currencies"

# No parameters requiered

currencies_df = pd.DataFrame(requests.get(coingecko + supported_currencies_ep).json()).rename(columns={0: "currencies"})

In [296]:
currencies_df[currencies_df["currencies"].str.contains("mx*")]

,currencies
7,xlm
17,bmd
36,mmk
37,mxn
38,myr


#### Get Data by ID or Contract

* Get the current coin's price

In [297]:
simple_price_ep = "simple/price"

coin_ids = ",".join(coin_list_df["id"].tolist())

simple_price_parameters = {
    "ids": coin_ids,
    "vs_currencies": "usd",
    "include_market_cap": "true",
    "include_24hr_vol": "true",
    "include_24hr_change": "true",
    "include_last_updated_at": "true",
    "precision": "4"
}

simple_price_of_coins_df = pd.DataFrame(requests.get(coingecko + simple_price_ep, params=simple_price_parameters).json()).T

In [298]:
simple_price_of_coins_df

,usd,usd_market_cap,usd_24h_vol,usd_24h_change,last_updated_at
apecoin,1.8315,6.743951e+08,1.076327e+08,-11.149230,1.692159e+09
cyberconnect,5.1745,5.714613e+07,1.152565e+08,-25.929567,1.692159e+09
harrypotterobamasonic10in,0.1747,1.759519e+08,1.134303e+07,12.774902,1.692159e+09
lybra-finance,2.2350,3.003692e+07,6.033030e+06,6.696455,1.692159e+09
sei-network,0.2465,4.408975e+08,3.011364e+08,24.437057,1.692159e+09
thorchain,1.6058,4.841066e+08,3.154982e+08,3.311657,1.692159e+09
worldcoin-wld,1.5867,2.002757e+08,4.691704e+07,-11.086433,1.692159e+09


#### Historical Data

`/coin/...` end points

        `/coins/{id}/history`

In [300]:
var = "hello"
f"some texts {var} here"

'some texts hello here'

Let's run a simple attempt to get a historical data from aby coin.. later, this will be run through all coins

In [323]:
coin_id = "apecoin"
market_chart_ep = f"coins/{coin_id}/market_chart"

market_chart_parameters = {
    "vs_currency": "usd",
    "days": "90",
    "interval": "daily",
    "precision": "4"
}

market_chart_coin_df = pd.DataFrame(requests.get(coingecko + market_chart_ep, params=market_chart_parameters).json())


In [324]:
market_chart_coin_df

,prices,market_caps,total_volumes
0,"[1684454400000, 3.3936]","[1684454400000, 1254051281.632525]","[1684454400000, 63729409.5808864]"
1,"[1684540800000, 3.4277]","[1684540800000, 1264859478.5641994]","[1684540800000, 49338082.112148724]"
2,"[1684627200000, 3.4444]","[1684627200000, 1269807769.3137863]","[1684627200000, 31133846.791591607]"
3,"[1684713600000, 3.4689]","[1684713600000, 1280253491.5564191]","[1684713600000, 54356186.896674365]"
4,"[1684800000000, 3.4331]","[1684800000000, 1265003219.9816499]","[1684800000000, 44440052.502822876]"
...,...,...,...
86,"[1691884800000, 2.0137]","[1691884800000, 740804194.9886361]","[1691884800000, 155449659.48348713]"
87,"[1691971200000, 2.003]","[1691971200000, 735512264.9558252]","[1691971200000, 107841022.50986096]"
88,"[1692057600000, 2.0577]","[1692057600000, 759791210.4290547]","[1692057600000, 128609570.57890883]"
89,"[1692144000000, 1.8437]","[1692144000000, 679107753.6536336]","[1692144000000, 105204639.42863622]"


It seems each column has its own timestamp, this timestamp is in UNIX.. so that we have to reshape this in order to get the dataframe ready for time series visualization

In [325]:
pd.to_datetime(1684454400000, unit="ms") # it worked

Timestamp('2023-05-19 00:00:00')

In [329]:
# Let's try this out.. 
pd.to_datetime(market_chart_coin_df["prices"].apply(lambda price: price[0]), unit="ms") 

0    2023-05-19 00:00:00
1    2023-05-20 00:00:00
2    2023-05-21 00:00:00
3    2023-05-22 00:00:00
4    2023-05-23 00:00:00
             ...        
86   2023-08-13 00:00:00
87   2023-08-14 00:00:00
88   2023-08-15 00:00:00
89   2023-08-16 00:00:00
90   2023-08-16 04:31:06
Name: prices, Length: 91, dtype: datetime64[ns]

In [330]:
dates = pd.to_datetime(market_chart_coin_df["prices"].apply(lambda price: price[0]), unit="ms")

In [331]:
market_chart_coin_df.set_index(dates)

,prices,market_caps,total_volumes
prices,,,
2023-05-19 00:00:00,"[1684454400000, 3.3936]","[1684454400000, 1254051281.632525]","[1684454400000, 63729409.5808864]"
2023-05-20 00:00:00,"[1684540800000, 3.4277]","[1684540800000, 1264859478.5641994]","[1684540800000, 49338082.112148724]"
2023-05-21 00:00:00,"[1684627200000, 3.4444]","[1684627200000, 1269807769.3137863]","[1684627200000, 31133846.791591607]"
2023-05-22 00:00:00,"[1684713600000, 3.4689]","[1684713600000, 1280253491.5564191]","[1684713600000, 54356186.896674365]"
2023-05-23 00:00:00,"[1684800000000, 3.4331]","[1684800000000, 1265003219.9816499]","[1684800000000, 44440052.502822876]"
...,...,...,...
2023-08-13 00:00:00,"[1691884800000, 2.0137]","[1691884800000, 740804194.9886361]","[1691884800000, 155449659.48348713]"
2023-08-14 00:00:00,"[1691971200000, 2.003]","[1691971200000, 735512264.9558252]","[1691971200000, 107841022.50986096]"
2023-08-15 00:00:00,"[1692057600000, 2.0577]","[1692057600000, 759791210.4290547]","[1692057600000, 128609570.57890883]"
